<a href="https://colab.research.google.com/github/roosbot/Matching-Supply-Demand/blob/main/supplier_deep_dive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Prep

In [ ]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# to retrieve data from drive
from google.colab import drive
drive.mount('/content/gdrive')

# to export data to sheets
import gspread
from google.colab import auth
from google.auth import default

auth.authenticate_user()

creds, _ = default()

gc = gspread.authorize(creds)

# link with Google Sheets
spreadsheet_name = "Supplier Deep Dive"
try:
    spreadsheet = gc.open(spreadsheet_name)
except gspread.SpreadsheetNotFound:
    print(f"Spreadsheet '{spreadsheet_name}' not found.")
    exit()

Mounted at /content/gdrive


# **items.csv**
- item_id: unique identifier for an item
- item_category: the type of supply being offered
- item_diet_category: if applicable, the dietary category of the item
- store_id: unique identifier for the store that is offering the given item
- store_segment: the category of store
- chain_id: if applicable, the unique identifier for the chain that the given store is a part of

In [ ]:
df_items = pd.read_csv('/content/gdrive/MyDrive/Work/Job Searching/Assesments/Too Good To Go/DataAnalyticsBusinessPartner_CaseStudy/Input/items.csv')
df_items.head(5)

,item_id,item_category,item_diet_category,store_id,store_segment,chain_id
0,b2f4863b540305b5cb0f343870cb5551,MEAL,NONE,5af373dd5a01fe5ae20c3fa22258f7bb,Cafeteria,NaN
1,e8d09f7f2603b30eece61acfeca2f1ce,MEAL,NONE,c50a028f58e273985ff1c62ee1722117,Fruits & vegetables store,NaN
2,26e944700c9937cf468fcea7a48e5e18,BAKED_GOODS,NONE,a1057d24a8983c7c2b64e88f77adcf5f,Cafeteria,NaN
3,08f3413188d18069d7108fbaffb45eb9,MEAL,NONE,05dc6b4cd1d240ca816e0bf9a1e0c2d4,Traditional Restaurant,e90acf987b0389430374a954671dc4bf
4,7e0d52915c0bf25447fee1b351792665,MEAL,NONE,63d557b5e98fba97f23faf88d694e2ef,Butcher,NaN


In [ ]:
# prompt: I want to create a boolean column called "is_chain" based on 'chain_id' where NaN values are marked 'False' and all non-NaN values True, use isnull() and not lambda please. Also show df_items.head(5) after for me to check the results

df_items['is_chain'] = ~df_items['chain_id'].isnull()
df_items.head(5)

,item_id,item_category,item_diet_category,store_id,store_segment,chain_id,is_chain
0,b2f4863b540305b5cb0f343870cb5551,MEAL,NONE,5af373dd5a01fe5ae20c3fa22258f7bb,Cafeteria,NaN,False
1,e8d09f7f2603b30eece61acfeca2f1ce,MEAL,NONE,c50a028f58e273985ff1c62ee1722117,Fruits & vegetables store,NaN,False
2,26e944700c9937cf468fcea7a48e5e18,BAKED_GOODS,NONE,a1057d24a8983c7c2b64e88f77adcf5f,Cafeteria,NaN,False
3,08f3413188d18069d7108fbaffb45eb9,MEAL,NONE,05dc6b4cd1d240ca816e0bf9a1e0c2d4,Traditional Restaurant,e90acf987b0389430374a954671dc4bf,True
4,7e0d52915c0bf25447fee1b351792665,MEAL,NONE,63d557b5e98fba97f23faf88d694e2ef,Butcher,NaN,False


In [ ]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23650 entries, 0 to 23649
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_id             23650 non-null  object
 1   item_category       23650 non-null  object
 2   item_diet_category  23650 non-null  object
 3   store_id            23650 non-null  object
 4   store_segment       23474 non-null  object
 5   chain_id            10934 non-null  object
 6   is_chain            23650 non-null  bool  
dtypes: bool(1), object(6)
memory usage: 1.1+ MB


In [ ]:
df_items.isnull().sum()

,0
item_id,0
item_category,0
item_diet_category,0
store_id,0
store_segment,176
chain_id,12716
is_chain,0


In [ ]:
# Fill all null values
df_items = df_items.fillna("(Not Available)")

In [ ]:
df_items.isnull().sum()

,0
item_id,0
item_category,0
item_diet_category,0
store_id,0
store_segment,0
chain_id,0
is_chain,0


In [ ]:
print("\nTotal unique items listed:", df_items['item_id'].nunique())
print("\nitem_category:",df_items["item_category"].unique())
print("\nitem_diet_category:",df_items["item_diet_category"].unique())
print("\nTotal unique stores:", df_items['store_id'].nunique())
print("\nTotal number of store segments:",df_items["store_segment"].nunique())
print("\nUnique store segments:",df_items["store_segment"].unique())
print("\nTotal number of chains :",df_items["chain_id"].nunique())
print("\nTotal number of non-chains :", df_items['store_id'].nunique()-df_items["chain_id"].nunique())


Total unique items listed: 23650

item_category: ['MEAL' 'BAKED_GOODS' 'GROCERIES' 'NON_FOOD' 'OTHER' 'ESSENTIAL_BAG']

item_diet_category: ['NONE' 'VEGETARIAN' 'VEGAN']

Total unique stores: 20005

Total number of store segments: 24

Unique store segments: ['Cafeteria' 'Fruits & vegetables store' 'Traditional Restaurant'
 'Butcher' 'Bakery' 'Supermarket Small' 'Pastry' 'Gas station' 'Fish shop'
 'Supermarket Medium' 'Fast Food / Take away restaurant' '(Not Available)'
 'Buffet' 'Hotel' 'Supermarket Big' 'Beverages' 'Manufacturer' 'Other'
 'Caterer' 'Florist' 'Other Specialists' 'Sushi' 'Market stall' 'Canteen']

Total number of chains : 1566

Total number of non-chains : 18439


In [ ]:
# Clean item names
df_items['item_category'] = df_items['item_category'].str.replace('_', ' ').str.title()
df_items['item_diet_category'] = df_items['item_diet_category'].str.replace('_', ' ').str.title()
df_items['store_segment'] = df_items['store_segment'].str.replace('_', ' ').str.title()

print("\nitem_category:",df_items["item_category"].unique())
print("\nitem_diet_category:",df_items["item_diet_category"].unique())
print("\nstore_segment:",df_items["store_segment"].unique())



item_category: ['Meal' 'Baked Goods' 'Groceries' 'Non Food' 'Other' 'Essential Bag']

item_diet_category: ['None' 'Vegetarian' 'Vegan']

store_segment: ['Cafeteria' 'Fruits & Vegetables Store' 'Traditional Restaurant'
 'Butcher' 'Bakery' 'Supermarket Small' 'Pastry' 'Gas Station' 'Fish Shop'
 'Supermarket Medium' 'Fast Food / Take Away Restaurant' '(Not Available)'
 'Buffet' 'Hotel' 'Supermarket Big' 'Beverages' 'Manufacturer' 'Other'
 'Caterer' 'Florist' 'Other Specialists' 'Sushi' 'Market Stall' 'Canteen']


# **purchases.csv**
- purchase_id: unique identifier for a purchase
- user_id: unique identifier for the user that completed the given purchase
- date: the date of the given purchase
- item_id: unique identifier for the item being purchased
- stock_count: the number of magic bags purchased

In [ ]:
df_purchases = pd.read_csv('/content/gdrive/MyDrive/Work/Job Searching/Assesments/Too Good To Go/DataAnalyticsBusinessPartner_CaseStudy/Input/purchases.csv')
df_purchases.head(5)

,purchase_id,user_id,date,item_id,stock_count
0,e48d15f0bc1cde91992a4acb9c8a476b,d1c580b8943a0c81fc705a87fc618c7f,2019-04-04,042aec9e604155f2f06c0a16c5f9ba06,1
1,f786f158f367b22a4e16175684d8a4e8,d1c580b8943a0c81fc705a87fc618c7f,2019-08-08,042aec9e604155f2f06c0a16c5f9ba06,1
2,4a7a804cc62b3d908027257ae5a62ba5,d1c580b8943a0c81fc705a87fc618c7f,2019-08-19,34b99cc2f738d8a132faea2e0cc4c13b,1
3,2e63a08be1199564edde68be3804ea93,d1c580b8943a0c81fc705a87fc618c7f,2019-08-27,a0f1a0686c4ecffbf795ea2d2a42817e,1
4,fff7d05fffb6ff79467b01384262ff90,d1c580b8943a0c81fc705a87fc618c7f,2019-11-22,67a5de9fa89738da0c6835ef457b5878,1


In [ ]:
df_purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5089605 entries, 0 to 5089604
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   purchase_id  object
 1   user_id      object
 2   date         object
 3   item_id      object
 4   stock_count  int64 
dtypes: int64(1), object(4)
memory usage: 194.2+ MB


In [ ]:
df_purchases['date'] = pd.to_datetime(df_purchases['date'])
df_purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5089605 entries, 0 to 5089604
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   purchase_id  object        
 1   user_id      object        
 2   date         datetime64[ns]
 3   item_id      object        
 4   stock_count  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 194.2+ MB


In [ ]:
# Check if date looks good still
df_purchases.head(5)

,purchase_id,user_id,date,item_id,stock_count
0,e48d15f0bc1cde91992a4acb9c8a476b,d1c580b8943a0c81fc705a87fc618c7f,2019-04-04,042aec9e604155f2f06c0a16c5f9ba06,1
1,f786f158f367b22a4e16175684d8a4e8,d1c580b8943a0c81fc705a87fc618c7f,2019-08-08,042aec9e604155f2f06c0a16c5f9ba06,1
2,4a7a804cc62b3d908027257ae5a62ba5,d1c580b8943a0c81fc705a87fc618c7f,2019-08-19,34b99cc2f738d8a132faea2e0cc4c13b,1
3,2e63a08be1199564edde68be3804ea93,d1c580b8943a0c81fc705a87fc618c7f,2019-08-27,a0f1a0686c4ecffbf795ea2d2a42817e,1
4,fff7d05fffb6ff79467b01384262ff90,d1c580b8943a0c81fc705a87fc618c7f,2019-11-22,67a5de9fa89738da0c6835ef457b5878,1


In [ ]:
df_purchases.isnull().sum()

# no critial rows like item_id are missing so ready to continue and no need for missing handling values

,0
purchase_id,0
user_id,0
date,0
item_id,0
stock_count,0


In [ ]:
print("\nTotal unique purchases:", df_purchases['purchase_id'].nunique())
print("\nTotal unique users:", df_purchases['user_id'].nunique())
print("\nTotal unique items purchased:", df_purchases['item_id'].nunique())
print("\nTotal bags purchased:", df_purchases['stock_count'].sum())


Total unique purchases: 5089605

Total unique users: 871230

Total unique items purchased: 23650

Total bags purchased: 5669861


# Exploration

In [ ]:
# Now merge the datasets as that is more useful going forward
df = pd.merge(df_items, df_purchases, on='item_id', how='inner')
df.head(5)

,item_id,item_category,item_diet_category,store_id,store_segment,chain_id,is_chain,purchase_id,user_id,date,stock_count
0,b2f4863b540305b5cb0f343870cb5551,Meal,None,5af373dd5a01fe5ae20c3fa22258f7bb,Cafeteria,(Not Available),False,b5ab69198f44f429fd0288393c4b51fe,56a8aa8f6a0d0cc77e6c688e1e6bf973,2021-05-10,1
1,b2f4863b540305b5cb0f343870cb5551,Meal,None,5af373dd5a01fe5ae20c3fa22258f7bb,Cafeteria,(Not Available),False,c29986a4c8e87e0bacb8d4f77d6607a7,e7c3f60adc8d736900fb2c146f9fd899,2021-05-29,1
2,b2f4863b540305b5cb0f343870cb5551,Meal,None,5af373dd5a01fe5ae20c3fa22258f7bb,Cafeteria,(Not Available),False,32ce7a492e15e18c4a8c0d700f8b0ab0,0025196ed96d129fc0e5e58667e277fe,2021-05-08,1
3,b2f4863b540305b5cb0f343870cb5551,Meal,None,5af373dd5a01fe5ae20c3fa22258f7bb,Cafeteria,(Not Available),False,ae8d16c4aaede1b369ea67b37ef16f76,0025196ed96d129fc0e5e58667e277fe,2021-05-14,1
4,b2f4863b540305b5cb0f343870cb5551,Meal,None,5af373dd5a01fe5ae20c3fa22258f7bb,Cafeteria,(Not Available),False,375dd9a7fd0e5f37393c77ad695351f6,0025196ed96d129fc0e5e58667e277fe,2021-05-22,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5089605 entries, 0 to 5089604
Data columns (total 11 columns):
 #   Column              Dtype         
---  ------              -----         
 0   item_id             object        
 1   item_category       object        
 2   item_diet_category  object        
 3   store_id            object        
 4   store_segment       object        
 5   chain_id            object        
 6   is_chain            bool          
 7   purchase_id         object        
 8   user_id             object        
 9   date                datetime64[ns]
 10  stock_count         int64         
dtypes: bool(1), datetime64[ns](1), int64(1), object(8)
memory usage: 393.2+ MB


In [ ]:
df.describe()

,date,stock_count
count,5089605,5.089605e+06
mean,2021-02-05 10:49:11.022407680,1.114008e+00
min,2019-01-01 00:00:00,1.000000e+00
25%,2020-09-26 00:00:00,1.000000e+00
50%,2021-03-30 00:00:00,1.000000e+00
75%,2021-08-24 00:00:00,1.000000e+00
max,2021-12-31 00:00:00,2.000000e+01
std,NaN,3.913120e-01


In [ ]:
print("\nstore_segment:",df["store_segment"].unique())
print("\nis_chain:",df["is_chain"].unique())
print("\nitem_category:",df["item_category"].unique())
print("\nitem_diet_category:",df["item_diet_category"].unique())



store_segment: ['Cafeteria' 'Fruits & Vegetables Store' 'Traditional Restaurant'
 'Butcher' 'Bakery' 'Supermarket Small' 'Pastry' 'Gas Station' 'Fish Shop'
 'Supermarket Medium' 'Fast Food / Take Away Restaurant' '(Not Available)'
 'Buffet' 'Hotel' 'Supermarket Big' 'Beverages' 'Manufacturer' 'Other'
 'Caterer' 'Florist' 'Other Specialists' 'Sushi' 'Market Stall' 'Canteen']

is_chain: [False  True]

item_category: ['Meal' 'Baked Goods' 'Groceries' 'Non Food' 'Other' 'Essential Bag']

item_diet_category: ['None' 'Vegetarian' 'Vegan']


In [ ]:
df['stock_count'].value_counts()

,count
stock_count,
1,4600119
2,424406
3,48185
4,11124
5,4363
6,789
7,247
8,146
10,104


In [ ]:
# Aggregate by month, store_segment, item_category, is_chain, item_diet_category

df['is_vegvegan'] = df['item_diet_category'].apply(lambda x: x == 'Vegan' or x == 'Vegetarian')


df_aggregated = df.groupby([pd.Grouper(key='date', freq='YE'), 'store_segment', 'item_category', 'is_chain', 'is_vegvegan']).agg(
    number_of_stores=('store_id', 'nunique'),
    number_of_purchases=('purchase_id', 'nunique'),
    number_of_bags_sold=('stock_count', 'sum')
).reset_index()

df_aggregated = df_aggregated.rename(columns={'date': 'year'})
df_aggregated['year'] = df_aggregated['year'].dt.strftime('%Y-%m-%d')

df_aggregated.head()

<ipython-input-37-e907c8300888>:6: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  df_aggregated = df.groupby([pd.Grouper(key='date', freq='Y'), 'store_segment', 'item_category', 'is_chain', 'is_vegvegan']).agg(


,year,store_segment,item_category,is_chain,is_vegvegan,number_of_stores,number_of_purchases,number_of_bags_sold
0,2019-12-31,(Not Available),Baked Goods,False,False,24,1123,1152
1,2019-12-31,(Not Available),Baked Goods,True,False,6,484,502
2,2019-12-31,(Not Available),Groceries,False,False,5,160,167
3,2019-12-31,(Not Available),Groceries,False,True,9,516,520
4,2019-12-31,(Not Available),Meal,False,False,60,1885,2092


In [ ]:
# Import all exploration data into Google Sheets for visualisation
data_queries = {
    "Aggregated data": df_aggregated,

}

empty_rows_to_leave = 3
current_row = 1
worksheet_name = "Data"

try:
    worksheet = spreadsheet.worksheet(worksheet_name)
except gspread.WorksheetNotFound:
    worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows=100, cols=20)
    print(f"Worksheet '{worksheet_name}' created.")

# Clear the entire worksheet before writing new data
worksheet.clear()

for sheet_name, data_df in data_queries.items(): # Renamed df to data_df to avoid shadowing the outer df
    print(f"Writing '{sheet_name}' starting at row {current_row}...")

    # Prepare data for writing (including headers)
    data_to_sheet = [data_df.columns.tolist()] + data_df.values.tolist()

    # Write the data
    worksheet.append_rows(data_to_sheet, value_input_option='USER_ENTERED', table_range=f'A{current_row}')

    # Update the starting row for the next dataset
    current_row += len(data_to_sheet) + empty_rows_to_leave

print("All data queries written to the Google Sheet.")

Writing 'Aggregated data' starting at row 1...
All data queries written to the Google Sheet.


# Analysis


In [ ]:
### ITEM CATEGORY

# Calculate number of stores per item category
stores_per_category = df.groupby('item_category')['store_id'].nunique().reset_index()
stores_per_category.columns = ['item_category', 'number_of_stores']

# Calculate number of purchases per item category
purchases_per_category = df.groupby('item_category')['purchase_id'].nunique().reset_index()
purchases_per_category.columns = ['item_category', 'number_of_purchases']

# Calculate number of bags sold per item category
bags_sold_per_category = df.groupby('item_category')['stock_count'].sum().reset_index()
bags_sold_per_category.columns = ['item_category', 'number_of_bags_sold']

# Group by 'item_category' and 'item_diet_category' and sum 'number_of_bags_sold'
bags_sold_by_category_diet = df.groupby(['item_category', 'item_diet_category'])['stock_count'].sum().reset_index()

# Pivot the table to have item_diet_category as columns
bags_sold_by_category_diet = bags_sold_by_category_diet.pivot(index='item_category', columns='item_diet_category', values='stock_count')

# Rename the columns for clarity
bags_sold_by_category_diet = bags_sold_by_category_diet.rename(columns={
    'None': 'number_bags_none',
    'Vegetarian': 'number_bags_veg',
    'Vegan': 'number_bags_vegan'
})

# Reset index to make 'item_category' a regular column again
bags_sold_by_category_diet = bags_sold_by_category_diet.reset_index()

float_cols = bags_sold_by_category_diet.select_dtypes(include='float64').columns

for col in float_cols:
    bags_sold_by_category_diet[col] = bags_sold_by_category_diet[col].astype('Int64')

bags_sold_by_category_diet = bags_sold_by_category_diet.fillna(0)

# Merge the dataframes to get all desired columns
supply_demand_by_category = pd.merge(stores_per_category, purchases_per_category, on='item_category', how='left')
supply_demand_by_category = pd.merge(supply_demand_by_category, bags_sold_per_category, on='item_category', how='left')
supply_demand_by_category = pd.merge(supply_demand_by_category, bags_sold_by_category_diet, on='item_category', how='left')

# Ensure all diet columns are present, even if there were no sales for a diet type across all categories
diet_columns = ['number_bags_none', 'number_bags_veg', 'number_bags_vegan']
for col in diet_columns:
    if col not in supply_demand_by_category.columns:
        supply_demand_by_category[col] = 0

# Add the new column 'number_of_bags_vegvegan'
# fillna(0) will handle missing values for calculation
supply_demand_by_category['number_of_bags_vegvegan'] = \
    supply_demand_by_category['number_bags_veg'].fillna(0) + \
    supply_demand_by_category['number_bags_vegan'].fillna(0)

# Select and reorder the desired columns
supply_demand_by_category = supply_demand_by_category[['item_category', 'number_of_stores', 'number_of_purchases', 'number_of_bags_sold', 'number_bags_none', 'number_bags_veg', 'number_bags_vegan','number_of_bags_vegvegan']]
supply_demand_by_category = supply_demand_by_category.sort_values(by='number_of_bags_sold', ascending=False)

supply_demand_by_category.head(5)

,item_category,number_of_stores,number_of_purchases,number_of_bags_sold,number_bags_none,number_bags_veg,number_bags_vegan,number_of_bags_vegvegan
3,Meal,10012,2134636,2444663,2408287,15074,21302,36376
0,Baked Goods,4012,1418355,1488650,1484718,1026,2906,3932
2,Groceries,5605,1229208,1392776,1240189,137576,15011,152587
5,Other,801,295005,330563,328265,1051,1247,2298
4,Non Food,95,12109,12912,12853,59,0,59


In [ ]:
### ITEM DIET CATEGORY

## SUPPLY
# Stores per diet item category: Which item diet category has the most stores?
stores_by_diet_category = df.groupby('item_diet_category')['store_id'].nunique().reset_index() # Use nunique for distinct store_ids
stores_by_diet_category.columns = ['item_diet_category', 'number_of_stores']
stores_by_diet_category = stores_by_diet_category.sort_values(by='number_of_stores', ascending=False)

## DEMAND
# Purchases by item diet category: How many times was each item_diet_category purchased?
purchases_by_diet_category = df.groupby('item_diet_category')['purchase_id'].nunique().reset_index() # Use nunique for distinct purchase_ids
purchases_by_diet_category.columns = ['item_diet_category', 'number_of_purchases']
purchases_by_diet_category = purchases_by_diet_category.sort_values(by='number_of_purchases', ascending=False)

# Calculate the number of magic bags sold per item diet category
bags_sold_by_diet_category = df.groupby('item_diet_category')['stock_count'].sum().reset_index()
bags_sold_by_diet_category.columns = ['item_diet_category', 'number_of_bags_sold']

# Merge supply and demand dataframes
supply_demand_by_diet_category = pd.merge(stores_by_diet_category, purchases_by_diet_category, on='item_diet_category', how='left')
supply_demand_by_diet_category = pd.merge(supply_demand_by_diet_category, bags_sold_by_diet_category, on='item_diet_category', how='left')

print("\nSupply and Demand characteristics by item diet category:")
supply_demand_by_diet_category


Supply and Demand characteristics by item diet category:


,item_diet_category,number_of_stores,number_of_purchases,number_of_bags_sold
0,None,19445,4906214,5474609
1,Vegetarian,566,146057,154786
2,Vegan,197,37334,40466


In [ ]:
### STORE SEGMENT & CHAIN
# Stores per store segment: Which store segment has the most stores?
stores_by_segment = df.groupby('store_segment')['store_id'].nunique().reset_index()
stores_by_segment.columns = ['store_segment', 'number_of_stores']
stores_by_segment = stores_by_segment.sort_values(by='number_of_stores', ascending=False)
total_stores = stores_by_segment['number_of_stores'].sum()
stores_by_segment['share_of_total_stores'] = stores_by_segment['number_of_stores'] / total_stores

# Purchases per store segment: Which store segment generated the most purchases?
purchases_by_segment = df.groupby('store_segment')['purchase_id'].nunique().reset_index()
purchases_by_segment.columns = ['store_segment', 'number_of_purchases']
purchases_by_segment = purchases_by_segment.sort_values(by='number_of_purchases', ascending=False)

# Calculate the number of magic bags sold per store segment
bags_sold_by_segment = df.groupby('store_segment')['stock_count'].sum().reset_index()
bags_sold_by_segment.columns = ['store_segment', 'number_of_bags_sold']

# Merge supply and demand dataframes
supply_demand_by_segment = pd.merge(stores_by_segment, purchases_by_segment, on='store_segment', how='left')
supply_demand_by_segment = pd.merge(supply_demand_by_segment, bags_sold_by_segment, on='store_segment', how='left')

# SEGMENT X CHAIN
# Group by both 'store_segment' and 'is_chain' to get counts for each combination
stores_by_segment_chain = df.groupby(['store_segment', 'is_chain'])['store_id'].nunique().reset_index()
stores_by_segment_chain.columns = ['store_segment', 'is_chain', 'number_of_stores']

# Group by both 'store_segment' and 'is_chain' to get purchase counts
purchases_by_segment_chain = df.groupby(['store_segment', 'is_chain'])['purchase_id'].nunique().reset_index()
purchases_by_segment_chain.columns = ['store_segment', 'is_chain', 'number_of_purchases']

# Group by both 'store_segment' and 'is_chain' to get bags sold counts
bags_sold_by_segment_chain = df.groupby(['store_segment', 'is_chain'])['stock_count'].sum().reset_index()
bags_sold_by_segment_chain.columns = ['store_segment', 'is_chain', 'number_of_bags_sold']

# Merge the three dataframes
supply_demand_by_segment_chain_intermediate = pd.merge(stores_by_segment_chain, purchases_by_segment_chain, on=['store_segment', 'is_chain'], how='left')
supply_demand_by_segment_chain_intermediate = pd.merge(supply_demand_by_segment_chain_intermediate, bags_sold_by_segment_chain, on=['store_segment', 'is_chain'], how='left')

# Reshape / reorder
supply_demand_by_segment_chain = supply_demand_by_segment_chain_intermediate.pivot_table(
    index='store_segment',
    columns='is_chain',
    values=['number_of_stores', 'number_of_purchases', 'number_of_bags_sold'],
    aggfunc='sum'
).reset_index()

supply_demand_by_segment_chain.columns = [
    'store_segment',
    'number_bags_nonchain', 'number_bags_chain',
    'number_purchases_nonchain', 'number_purchases_chain',
    'number_stores_nonchain', 'number_stores_chain'
]

# Merge the supply_demand_by_segment with the pivoted data
supply_demand_by_segment_chain = pd.merge(supply_demand_by_segment, supply_demand_by_segment_chain, on='store_segment', how='left')

# Reorder columns to the specified order
supply_demand_by_segment_chain = supply_demand_by_segment_chain[[
    'store_segment',
    'number_of_stores',
    'share_of_total_stores',
    'number_of_purchases',
    'number_of_bags_sold',
    'number_stores_chain',
    'number_stores_nonchain',
    'number_purchases_chain',
    'number_purchases_nonchain',
    'number_bags_chain',
    'number_bags_nonchain'
]]

## Add size for bubble chart visualisation
supply_demand_by_segment_chain['size'] = 1
supply_demand_by_segment_chain.loc[supply_demand_by_segment_chain['store_segment'] == 'Bakery', 'size'] = 2


supply_demand_by_segment_chain.head(5)

,store_segment,number_of_stores,share_of_total_stores,number_of_purchases,number_of_bags_sold,number_stores_chain,number_stores_nonchain,number_purchases_chain,number_purchases_nonchain,number_bags_chain,number_bags_nonchain,size
0,Bakery,3611,0.180505,1235802,1292319,1819,1792,848735,387067,889598,402721,2
1,Fruits & Vegetables Store,2821,0.141015,454155,472898,856,1965,193981,260174,203387,269511,1
2,Fast Food / Take Away Restaurant,2486,0.124269,641650,749030,844,1642,279127,362523,323678,425352,1
3,Traditional Restaurant,1885,0.094226,322720,377681,374,1511,111405,211315,130003,247678,1
4,Cafeteria,1772,0.088578,427866,453275,550,1222,294164,133702,310933,142342,1


In [ ]:
# prompt: Can you create a dataframe that shows the store_segment for each row and for each value in item_category show the amount of stores in separate columns. Make sure the numbers are integers and not float64. Make sure the order of store segments is descending based on total stores

# Pivot the data to get item_category as columns and count unique store_ids
supply_by_category = df.groupby(['store_segment', 'item_category'])['store_id'].nunique().unstack(fill_value=0)

# Add a total column to sort by
supply_by_category['Total Stores'] = supply_by_category.sum(axis=1)

# Sort the DataFrame by the total number of stores in descending order
supply_by_category = supply_by_category.sort_values(by='Total Stores', ascending=False)

# Drop the temporary 'Total Stores' column
supply_by_category = supply_by_category.drop(columns=['Total Stores'])

# Convert all columns to integer type
for col in supply_by_category.columns:
    supply_by_category[col] = supply_by_category[col].astype(int)

# Reset index to make 'store_segment' a column
supply_by_category = supply_by_category.reset_index()

supply_by_category.head(5)

item_category,store_segment,Baked Goods,Essential Bag,Groceries,Meal,Non Food,Other
0,Bakery,2506,1,62,1118,1,13
1,Fruits & Vegetables Store,8,1,1704,1107,2,49
2,Fast Food / Take Away Restaurant,64,0,41,2359,0,43
3,Traditional Restaurant,41,0,38,1815,0,16
4,Cafeteria,770,0,61,966,1,39


In [ ]:
## MONTHLY STORES AND BAGS BY CHAIN / DIET / ITEM_CATEGORY

# Aggregate data by month and is_chain status
df_monthly = df.groupby([df['date'].dt.to_period('M'), 'is_chain']).agg(
    number_of_bags_sold=('stock_count', 'sum'),
    number_of_stores=('store_id', 'nunique') # Count unique stores per month per chain status
).reset_index()

# Pivot the table to have is_chain as columns for number_of_stores and number_of_bags_sold
df_monthly_pivot = df_monthly.pivot_table(
    index='date',
    columns='is_chain',
    values=['number_of_bags_sold', 'number_of_stores'],
    aggfunc='sum' # Sum in case a store appears in both True and False chains within a month (though this shouldn't happen)
).reset_index()

# Flatten the MultiIndex columns
df_monthly_pivot.columns = [f'{col[0]}_{col[1]}' if col[1] in [True, False] else col[0] for col in df_monthly_pivot.columns]

# Rename the columns to the desired format
df_monthly_pivot = df_monthly_pivot.rename(columns={
    'number_of_stores_True': 'number_stores_chain',
    'number_of_stores_False': 'number_stores_nonchain',
    'number_of_bags_sold_False': 'number_of_bags_none', # Assuming non-chain corresponds to 'None' diet category for simplification here, though this isn't strictly correct based on previous logic. We'll need to refine bag counts by diet category per month.
    'number_of_bags_sold_True': 'number_of_bags_chain_total'
})

# To correctly get the bag counts by diet category per month,
# we need to group by month and item_diet_category
df_monthly_diet_bags = df.groupby([df['date'].dt.to_period('M'), 'item_diet_category']).agg(
    monthly_bags_sold=('stock_count', 'sum')
).reset_index()

# Pivot this table to get diet categories as columns
df_monthly_diet_bags_pivot = df_monthly_diet_bags.pivot_table(
    index='date',
    columns='item_diet_category',
    values='monthly_bags_sold',
    aggfunc='sum'
).reset_index()

# Rename the diet category columns
df_monthly_diet_bags_pivot.columns = [f'number_of_bags_{col.lower().replace(" ", "")}' if col != 'date' else col for col in df_monthly_diet_bags_pivot.columns]

# Ensure 'None', 'Vegetarian', 'Vegan' columns exist, fill missing with 0
diet_cols_needed = ['number_of_bags_none', 'number_of_bags_vegetarian', 'number_of_bags_vegan']
for col in diet_cols_needed:
    if col not in df_monthly_diet_bags_pivot.columns:
        df_monthly_diet_bags_pivot[col] = 0

# Calculate the total bags sold for veg and vegan per month
df_monthly_diet_bags_pivot['number_of_bags_vegvegan'] = \
    df_monthly_diet_bags_pivot['number_of_bags_vegetarian'] + \
    df_monthly_diet_bags_pivot['number_of_bags_vegan']

# Merge the store counts by chain status with the bag counts by diet category
monthly_supply_demand = pd.merge(
    df_monthly_pivot[['date', 'number_stores_chain', 'number_stores_nonchain']],
    df_monthly_diet_bags_pivot[['date', 'number_of_bags_none', 'number_of_bags_vegvegan']],
    on='date',
    how='left'
)

# Convert 'date' column to the specified YYYY-MM-DD format
monthly_supply_demand['date'] = monthly_supply_demand['date'].astype(str) + '-01' # Add day '01'
monthly_supply_demand['date'] = pd.to_datetime(monthly_supply_demand['date']).dt.strftime('%Y-%m-%d')

# Rename the date column to represent the month
monthly_supply_demand = monthly_supply_demand.rename(columns={'date': 'Month'})

# Fill any potential missing values after merges (e.g., if a month had no sales)
monthly_supply_demand = monthly_supply_demand.fillna(0)

# Convert appropriate columns to integer type
int_cols = ['number_stores_chain', 'number_stores_nonchain', 'number_of_bags_none', 'number_of_bags_vegvegan']
for col in int_cols:
    monthly_supply_demand[col] = monthly_supply_demand[col].astype(int)


monthly_supply_demand.head(5)

,Month,number_stores_chain,number_stores_nonchain,number_of_bags_none,number_of_bags_vegvegan
0,2019-01-01,261,117,6032,1209
1,2019-02-01,264,160,11669,1751
2,2019-03-01,363,240,19053,2022
3,2019-04-01,410,314,24339,2958
4,2019-05-01,500,447,31573,3640


In [ ]:
### MONTHLY STORES AND BAGS BY CHAIN / ITEM_CATEGORY / DIET_CATEGORY

# --- CHAIN ---
# Aggregate data by month and is_chain status
df_monthly = df.groupby([df['date'].dt.to_period('M'), 'is_chain']).agg(
    number_of_bags_sold=('stock_count', 'sum'),
    number_of_stores=('store_id', 'nunique') # Count unique stores per month per chain status
).reset_index()

# Pivot the table to have is_chain as columns for number_of_bags_sold and number_of_stores
df_monthly_pivot = df_monthly.pivot_table(
    index='date',
    columns='is_chain',
    values=['number_of_bags_sold', 'number_of_stores'],
    aggfunc='sum'
).reset_index()

# Flatten the MultiIndex columns
# This part ensures that 'number_of_bags_sold_True', 'number_of_stores_False', etc. are created
df_monthly_pivot.columns = [f'{col[0]}_{col[1]}' if col[1] in [True, False] else col[0] for col in df_monthly_pivot.columns]

# Rename the columns for chain/non-chain stores
df_monthly_pivot = df_monthly_pivot.rename(columns={
    'number_of_stores_True': 'number_stores_chain',
    'number_of_stores_False': 'number_stores_nonchain',
    'number_of_bags_sold_True': 'total_bags_chain_sales',
    'number_of_bags_sold_False': 'total_bags_nonchain_sales'})

# Calculate total number of stores per month (sum of chain and non-chain)
df_monthly_pivot['number_of_stores_total'] = df_monthly_pivot['number_stores_chain'] + df_monthly_pivot['number_stores_nonchain']

# --- ITEM_CATEGORY ---
# Group by month and item_category to get store counts and bag counts per category
df_monthly_item_category = df.groupby([df['date'].dt.to_period('M'), 'item_category']).agg(
    stores_per_category=('store_id', 'nunique'),
    bags_per_category=('stock_count', 'sum')
).reset_index()

# Pivot for stores per item category
df_monthly_stores_by_category = df_monthly_item_category.pivot_table(
    index='date',
    columns='item_category',
    values='stores_per_category',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Rename store columns for item categories
df_monthly_stores_by_category.columns = [
    f'number_of_stores_{col.lower().replace(" ", "_")}' if col != 'date' else col
    for col in df_monthly_stores_by_category.columns
]

# Ensure all expected item category store columns exist and fill with 0
expected_store_cols = [
    'number_of_stores_baked_goods',
    'number_of_stores_essential_bag',
    'number_of_stores_groceries',
    'number_of_stores_meal',
    'number_of_stores_non_food',
    'number_of_stores_other'
]
for col in expected_store_cols:
    if col not in df_monthly_stores_by_category.columns:
        df_monthly_stores_by_category[col] = 0

# --- DIET CATEGORY ---
df_monthly_diet_bags = df.groupby([df['date'].dt.to_period('M'), 'item_diet_category']).agg(
    monthly_bags_sold=('stock_count', 'sum')
).reset_index()

# Pivot this table to get diet categories as columns
df_monthly_diet_bags_pivot = df_monthly_diet_bags.pivot_table(
    index='date',
    columns='item_diet_category',
    values='monthly_bags_sold',
    aggfunc='sum',
    fill_value=0 # Fill missing with 0
).reset_index()

# Rename the diet category columns
df_monthly_diet_bags_pivot.columns = [f'number_of_bags_{col.lower().replace(" ", "")}' if col != 'date' else col for col in df_monthly_diet_bags_pivot.columns]

# Ensure 'None', 'Vegetarian', 'Vegan' columns exist, fill missing with 0
diet_cols_needed = ['number_of_bags_none', 'number_of_bags_vegetarian', 'number_of_bags_vegan']
for col in diet_cols_needed:
    if col not in df_monthly_diet_bags_pivot.columns:
        df_monthly_diet_bags_pivot[col] = 0

# Calculate the total bags sold for veg and vegan per month
df_monthly_diet_bags_pivot['number_of_bags_vegvegan'] = \
    df_monthly_diet_bags_pivot['number_of_bags_vegetarian'] + \
    df_monthly_diet_bags_pivot['number_of_bags_vegan']

# --- Merging all aggregated data into the final DataFrame ---
monthly_supply_demand = pd.merge(
    df_monthly_pivot[['date', 'number_stores_chain', 'number_stores_nonchain', 'number_of_stores_total']],
    df_monthly_stores_by_category, # Merge item category store counts
    on='date',
    how='left'
)

monthly_supply_demand = pd.merge(
    monthly_supply_demand,
    df_monthly_diet_bags_pivot[['date', 'number_of_bags_none', 'number_of_bags_vegvegan']],
    on='date',
    how='left'
)

# Convert 'date' column to the specified YYYY-MM-DD format
monthly_supply_demand['date'] = monthly_supply_demand['date'].astype(str) + '-01' # Add day '01'
monthly_supply_demand['date'] = pd.to_datetime(monthly_supply_demand['date']).dt.strftime('%Y-%m-%d')

# Rename the date column to represent the month
monthly_supply_demand = monthly_supply_demand.rename(columns={'date': 'Month'})

# Fill any potential missing values after merges (e.g., if a month had no sales for a specific category)
monthly_supply_demand = monthly_supply_demand.fillna(0)

# Convert appropriate columns to integer type (all count-based columns)
# Dynamically get all columns except 'Month' for conversion
int_cols_to_convert = [col for col in monthly_supply_demand.columns if col != 'Month']
for col in int_cols_to_convert:
    monthly_supply_demand[col] = monthly_supply_demand[col].astype(int)

# Final column reordering as per your request
final_columns_order = [
    'Month',
    'number_stores_chain',
    'number_stores_nonchain',
    'number_of_stores_baked_goods',
    'number_of_stores_essential_bag',
    'number_of_stores_groceries',
    'number_of_stores_meal',
    'number_of_stores_non_food',
    'number_of_stores_other',
    'number_of_stores_total', # Added this for a total store count
    'number_of_bags_none',
    'number_of_bags_vegvegan',
]

# Reindex to ensure the final order, filling with 0 if any column somehow didn't exist
# (though the 'ensure expected cols exist' steps should prevent this)
monthly_supply_demand = monthly_supply_demand.reindex(columns=final_columns_order, fill_value=0)

monthly_supply_demand.head(5)

,Month,number_stores_chain,number_stores_nonchain,number_of_stores_baked_goods,number_of_stores_essential_bag,number_of_stores_groceries,number_of_stores_meal,number_of_stores_non_food,number_of_stores_other,number_of_stores_total,number_of_bags_none,number_of_bags_vegvegan
0,2019-01-01,261,117,123,0,45,172,0,38,378,6032,1209
1,2019-02-01,264,160,124,0,46,212,0,42,424,11669,1751
2,2019-03-01,363,240,178,0,62,305,0,58,603,19053,2022
3,2019-04-01,410,314,227,0,72,354,0,72,724,24339,2958
4,2019-05-01,500,447,294,0,90,433,0,131,947,31573,3640


In [ ]:
# Step 1: Aggregate stock_count by store_segment and item_category
demand_segment_category = df.groupby(['store_segment', 'item_category'])['stock_count'].sum().reset_index()
demand_segment_category.rename(columns={'stock_count': 'number_of_bags_sold'}, inplace=True)

# Step 2: Pivot the aggregated data
demand_segment_category = demand_segment_category.pivot_table(
    index='store_segment',
    columns='item_category',
    values='number_of_bags_sold',
    fill_value=0
).reset_index()

# Step 3: Convert float columns to integers
for col in demand_segment_category.columns:
    if demand_segment_category[col].dtype == 'float64':
        demand_segment_category[col] = demand_segment_category[col].astype('int64')

# Step 4: Calculate total stock_count for each store_segment and sort
# Identify columns that represent item categories (all except 'store_segment')
item_category_cols = [col for col in demand_segment_category.columns if col != 'store_segment']

# Calculate the sum of stock_count across all item categories for each segment
demand_segment_category['total_segment_stock'] = demand_segment_category[item_category_cols].sum(axis=1)

# Sort the DataFrame by this new total column in descending order
demand_segment_category = demand_segment_category.sort_values(by='total_segment_stock', ascending=False)
# demand_segment_category = demand_segment_category.drop(columns=['total_segment_stock'])

# Step 5: Calculate the percentage share of stock for each item_category within its store_segment
for col in item_category_cols:
  # Avoid division by zero if total_segment_stock is 0
  demand_segment_category[f'{col}_share'] = (
      demand_segment_category[col] / demand_segment_category['total_segment_stock']
  ).fillna(0).round(3) # Fill NaN with 0 and round to 3 decimal places

demand_segment_category.head()

item_category,store_segment,Baked Goods,Essential Bag,Groceries,Meal,Non Food,Other,total_segment_stock,Baked Goods_share,Essential Bag_share,Groceries_share,Meal_share,Non Food_share,Other_share
1,Bakery,1008796,1,14256,266353,18,2895,1292319,0.781,0.0,0.011,0.206,0.0,0.002
20,Supermarket Medium,2586,0,578264,75931,0,126213,782994,0.003,0.0,0.739,0.097,0.0,0.161
8,Fast Food / Take Away Restaurant,13200,0,7453,712688,0,15689,749030,0.018,0.0,0.010,0.951,0.0,0.021
11,Fruits & Vegetables Store,1169,1,308736,147290,62,15640,472898,0.002,0.0,0.653,0.311,0.0,0.033
5,Cafeteria,309827,0,9627,125223,1,8597,453275,0.684,0.0,0.021,0.276,0.0,0.019


In [ ]:
## SEGMENT x ITEM CATEGORY x SALES

# Step 1: Calculate Segment-Category Bags Sold (Demand)
segment_category_demand = df.groupby(['store_segment', 'item_category'])['stock_count'].sum().reset_index()
segment_category_demand.rename(columns={'stock_count': 'total_bags_sold_for_category_in_segment'}, inplace=True)

# Step 2: Calculate Segment-Category Store Count (Supply)
segment_category_supply = df.groupby(['store_segment', 'item_category'])['store_id'].nunique().reset_index()
segment_category_supply.rename(columns={'store_id': 'number_of_stores_offering_category_in_segment'}, inplace=True)

# Step 3: Calculate Total Bags Sold per Segment (for percentage calculation)
total_bags_per_segment = df.groupby('store_segment')['stock_count'].sum().reset_index()
total_bags_per_segment.rename(columns={'stock_count': 'total_bags_sold_for_segment'}, inplace=True)

# --- Combine Data for Analysis ---
supply_demand_segment_category = pd.merge(segment_category_demand, segment_category_supply, on=['store_segment', 'item_category'], how='left')
supply_demand_segment_category = pd.merge(supply_demand_segment_category, total_bags_per_segment, on='store_segment', how='left')



# Step 4: Calculate Percentage of Segment Sales
supply_demand_segment_category['percentage_of_segment_sales'] = (supply_demand_segment_category['total_bags_sold_for_category_in_segment'] / supply_demand_segment_category['total_bags_sold_for_segment'])

# Step 5: Calculate Demand_Per_Store_Ratio
# Handle cases where 'number_of_stores_offering_category_in_segment' might be 0
supply_demand_segment_category['demand_per_store_ratio'] = supply_demand_segment_category.apply(
    lambda row: row['total_bags_sold_for_category_in_segment'] / row['number_of_stores_offering_category_in_segment']
    if row['number_of_stores_offering_category_in_segment'] > 0 else np.inf, # Use np.inf for truly unmet demand
    axis=1
)

supply_demand_segment_category.head(5)

,store_segment,item_category,total_bags_sold_for_category_in_segment,number_of_stores_offering_category_in_segment,total_bags_sold_for_segment,percentage_of_segment_sales,demand_per_store_ratio
0,(Not Available),Baked Goods,4726,30,27612,0.171157,157.533333
1,(Not Available),Groceries,1935,14,27612,0.070078,138.214286
2,(Not Available),Meal,13425,79,27612,0.486202,169.936709
3,(Not Available),Non Food,16,1,27612,0.000579,16.000000
4,(Not Available),Other,7510,35,27612,0.271983,214.571429


In [ ]:
## SEGMENT x ITEM CATEGORY x SALES

# Calculate segment-category bags sold (Demand)
segment_category_demand = df.groupby(['store_segment', 'item_category'])['stock_count'].sum().reset_index()
segment_category_demand.rename(columns={'stock_count': 'total_bags_sold_for_category_in_segment'}, inplace=True)

# Calculate segment-category store count (Supply)
segment_category_supply = df.groupby(['store_segment', 'item_category'])['store_id'].nunique().reset_index()
segment_category_supply.rename(columns={'store_id': 'number_of_stores_offering_category_in_segment'}, inplace=True)

# --- Combine Data for Analysis ---
supply_demand_segment_category = pd.merge(segment_category_demand, segment_category_supply, on=['store_segment', 'item_category'], how='left')

# --- Sort data ---
supply_demand_segment_category = supply_demand_segment_category.sort_values(
    by=['store_segment', 'total_bags_sold_for_category_in_segment'],
    ascending=[True, False] # Sort segments alphabetically, then ratio descending
)

supply_demand_segment_category.head(5)

,store_segment,item_category,total_bags_sold_for_category_in_segment,number_of_stores_offering_category_in_segment
2,(Not Available),Meal,13425,79
4,(Not Available),Other,7510,35
0,(Not Available),Baked Goods,4726,30
1,(Not Available),Groceries,1935,14
3,(Not Available),Non Food,16,1


In [ ]:
# Import all exploration data into Google Sheets for visualisation
data_queries = {
    "Supply and demand by item category": supply_demand_by_category,
    "Supply and demand by item diet category": supply_demand_by_diet_category,
    "Supply and demand by store segment and chain": supply_demand_by_segment_chain,
    "Supply by category": supply_by_category,
    "Supply and demand by month:" : monthly_supply_demand,
    "Demand by store segment and item category": demand_segment_category,
    "Supply and demand by store segment and item category": supply_demand_segment_category,
    # "Supply and demand by store segment": supply_demand_by_segment,
    # "Demand over time (monthly)": demand_over_time
    # "Supply and demand by store segment": avg_supply_demand_by_segment,

}

empty_rows_to_leave = 3
current_row = 1
worksheet_name = "Exploration"

try:
    worksheet = spreadsheet.worksheet(worksheet_name)
except gspread.WorksheetNotFound:
    worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows=100, cols=20)
    print(f"Worksheet '{worksheet_name}' created.")

# Clear the entire worksheet before writing new data
worksheet.clear()

for sheet_name, data_df in data_queries.items(): # Renamed df to data_df to avoid shadowing the outer df
    print(f"Writing '{sheet_name}' starting at row {current_row}...")

    # Prepare data for writing (including headers)
    data_to_sheet = [data_df.columns.tolist()] + data_df.values.tolist()

    # Write the data
    worksheet.append_rows(data_to_sheet, value_input_option='USER_ENTERED', table_range=f'A{current_row}')

    # Update the starting row for the next dataset
    current_row += len(data_to_sheet) + empty_rows_to_leave

print("All data queries written to the Google Sheet.")

Writing 'Supply and demand by item category' starting at row 1...
Writing 'Supply and demand by item diet category' starting at row 11...
Writing 'Supply and demand by store segment and chain' starting at row 18...
Writing 'Supply by category' starting at row 46...
Writing 'Supply and demand by month:' starting at row 74...
Writing 'Demand by store segment and item category' starting at row 114...
Writing 'Supply and demand by store segment and item category' starting at row 142...
All data queries written to the Google Sheet.


# Appendix

In [ ]:
# Aggregate by month and item category
monthly_category_purchases = merged_df.groupby([pd.Grouper(key='date', freq='M'), 'item_category']).size().reset_index(name='purchase_count')

# Sort for easier trend analysis
monthly_category_purchases = monthly_category_purchases.sort_values(by=['item_category', 'date'])

In [ ]:
# Purchase frequency distribution
purchases_per_store_segment = df.groupby(['store_segment', 'store_id'])['purchase_id'].nunique().reset_index()
avg_purchases_per_store_segment = purchases_per_store_segment.groupby('store_segment')['purchase_id'].mean().reset_index()
avg_purchases_per_store_segment.columns = ['store_segment', 'avg_purchases_per_store']
# print("\nAverage Purchase Frequency per Store by Segment:")
# print(avg_purchases_per_store_segment)

# Merge with the existing supply_demand_by_segment
avg_supply_demand_by_segment = pd.merge(supply_demand_by_segment, avg_purchases_per_store_segment, on='store_segment', how='left')


# Average bags purchased per store
avg_bags_per_purchase_segment = df.groupby(['store_segment', 'purchase_id'])['stock_count'].sum().reset_index()
avg_bags_per_purchase_segment = avg_bags_per_purchase_segment.groupby('store_segment')['stock_count'].mean().reset_index()
avg_bags_per_purchase_segment.columns = ['store_segment', 'avg_bags_per_purchase']
# print("\nAverage bags purchased per store:")
# print(avg_bags_per_purchase_segment)

# Merge with the existing supply_demand_by_segment
avg_supply_demand_by_segment = pd.merge(avg_supply_demand_by_segment, avg_bags_per_purchase_segment, on='store_segment', how='left')

print("\nUpdated Supply and Demand characteristics by store segment:")
avg_supply_demand_by_segment

In [ ]:
# Demand over time (e.g., monthly)
df['month'] = df['date'].dt.to_period('M')

purchases_over_time = df.groupby('month')['purchase_id'].count().reset_index()
bags_sold_over_time = df.groupby('month')['stock_count'].sum()

# Merge the two dataframes
demand_over_time = pd.merge(purchases_over_time, bags_sold_over_time, on='month', how='left')
demand_over_time.columns = ['month','number_of_purchases', 'number_of_bags_sold']


print("\nTotal Demand Over Time (Monthly):")
print(demand_over_time)

In [ ]:
# Supply by segment and category
demand_segment_category = df.groupby(['store_segment', 'item_category'])['store_id'].nunique().unstack().fillna(0)
# print("\nDemand by Store Segment and Item Category:")
print(demand_segment_category)

In [ ]:
# Demand by segment and category
demand_segment_category = df.groupby(['store_segment', 'item_category'])['stock_count'].sum().unstack().fillna(0)
# print("\nDemand by Store Segment and Item Category:")
print(demand_segment_category)

In [ ]:
Create a piece of code for each 'store_segment' and the columns column called 'demand_supply_ratio' and calculate it by dividing

In [ ]:
What if..

Sales rep can opens up their email or slack and get this automated message/suggestion?
-- Suggest a segment and within that segment suggest item categories and item diet categories
-- e.g. within bakeries you should focus on meals or vegetarian etc.
read more here..

Start with main insights, then dive into it
Define what is demand and what is supply (what data do we use currently - exhaustive )
Check whether one store cn have multuple item categories
Start with limitations, what are looking at and what aren't we looking at yet

Start with an example

Bakery --> 10% of stores bakery, 1% of sales is from bakery = balance
Supermarkt medium
Restaurant
Florist


opportunity sizing
doen we het al goed? overall 102% wat betekent dit


eindig met het grote doel = 80,000


Average Purchase Volume per Segment (type winkel) -- bakker vrekoopt 4,5K bags per jaar, ah 10K per jaar
Growth Trends / Seasonality -->
Item Categories within Segments

X% more bakeries, means X% more bags

In december the demand is high for X so then we need to add more supply?


Within bakeries, look at diets


Focus:
Suggest a segment and within that segment suggest item categories and item diet categories
e.g. within bakeries you should focus on meals or vegetarian etc.


-- look at average sales per store (for high impact but 'low' effort)
-- show data over time (to see low demand/lowsupply changing to high demand high supply for example)

In [ ]:
# df.to_csv('/content/gdrive/MyDrive/Work/Job Searching/Assesments/Too Good To Go/DataAnalyticsBusinessPartner_CaseStudy/Output/data.csv', index=False)

In [ ]:
# Calculate the total number of unique item_id's listed
total_listed_items = df_items['item_id'].nunique()
print(f"Total unique items listed: {total_listed_items}")

# Calculate the total number of unique item_id's purchased
total_purchased_items = df_purchases['item_id'].nunique()
print(f"Total unique items purchased: {total_purchased_items}")

# Calculate the number of unique item_id's purchased AND listed in items.csv
# This is the number of unique item_id's present in the 'item_id' column of the merged dataframe (df)
purchased_and_listed_items = df['item_id'].nunique()
print(f"Unique items purchased and listed in items.csv: {purchased_and_listed_items}")

# Calculate Demand Fulfillment Rate
if total_purchased_items > 0:
  df['Demand Fulfillment Rate'] = (purchased_and_listed_items / total_purchased_items) * 100
else:
  df['Demand Fulfillment Rate'] = 0 # Avoid division by zero if no items were purchased

# print("\nDataFrame with 'Demand Fulfillment Rate' column:")
# print(df[['item_id', 'purchase_id', 'Demand Fulfillment Rate']].head())

# If you want the overall Demand Fulfillment Rate for the entire dataset,
# you can access it from any row since it's calculated based on the whole dataframe
overall_demand_fulfillment_rate = df['Demand Fulfillment Rate'].iloc[0] if not df.empty else 0
print(f"\nOverall Demand Fulfillment Rate: {overall_demand_fulfillment_rate:.2f}%")

In [ ]:
### ITEM CATEGORY - AVG

# Purchase frequency distribution
purchases_per_category = df.groupby(['item_category'])['purchase_id'].nunique().reset_index()
avg_purchases_per_category = purchases_per_category.groupby('item_category')['purchase_id'].mean().reset_index()
avg_purchases_per_category.columns = ['item_category', 'avg_purchases_per_store']
# print("\nAverage purchase frequency per store by category:")
# print(avg_purchases_per_item_category)

# Merge with the existing supply_demand_by_category
avg_supply_demand_by_category = pd.merge(supply_demand_by_category, avg_purchases_per_category, on='item_category', how='left')


# Average bags purchased per store
avg_bags_per_purchase_category = df.groupby(['item_category', 'purchase_id'])['stock_count'].sum().reset_index()
avg_bags_per_purchase_category = avg_bags_per_purchase_category.groupby('item_category')['stock_count'].mean().reset_index()
avg_bags_per_purchase_category.columns = ['item_category', 'avg_bags_per_purchase']
# print("\nAverage bags purchased per store:")
# print(avg_bags_per_purchase_category)

# Merge with the existing supply_demand_by_category
avg_supply_demand_by_category = pd.merge(avg_supply_demand_by_category, avg_bags_per_purchase_category, on='item_category', how='left')

avg_supply_demand_by_category.head(5)

,item_category,number_of_stores,number_of_purchases,number_of_bags_sold,number_bags_none,number_bags_veg,number_bags_vegan,number_of_bags_vegvegan,avg_purchases_per_store,avg_bags_per_purchase
0,Meal,10012,2134636,2444663,2408287,15074,21302,36376,2134636.0,1.145236
1,Baked Goods,4012,1418355,1488650,1484718,1026,2906,3932,1418355.0,1.049561
2,Groceries,5605,1229208,1392776,1240189,137576,15011,152587,1229208.0,1.133068
3,Other,801,295005,330563,328265,1051,1247,2298,295005.0,1.120534
4,Non Food,95,12109,12912,12853,59,0,59,12109.0,1.066314


In [ ]:
# --- STORE_SEGMENT ---
# Group by month and segment to get store counts and bag counts per category
df_monthly_segment = df.groupby([df['date'].dt.to_period('M'), 'store_segment']).agg(
    stores_by_segment=('store_id', 'nunique'),
    bags_per_category=('stock_count', 'sum')
).reset_index()

# Pivot for stores by segment
df_monthly_stores_by_segment = df_monthly_segment.pivot_table(
    index='date',
    columns='store_segment',
    values='stores_by_segment',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Rename store columns for item categories
df_monthly_stores_by_segment.columns = [
    f'number_of_stores_{col.lower().replace(" ", "_")}' if col != 'date' else col
    for col in df_monthly_stores_by_segment.columns
]
df_monthly_stores_by_segment.head()

,date,number_of_stores_(not_available),number_of_stores_bakery,number_of_stores_beverages,number_of_stores_buffet,number_of_stores_butcher,number_of_stores_cafeteria,number_of_stores_canteen,number_of_stores_caterer,number_of_stores_fast_food_/_take_away_restaurant,...,number_of_stores_manufacturer,number_of_stores_market_stall,number_of_stores_other,number_of_stores_other_specialists,number_of_stores_pastry,number_of_stores_supermarket_big,number_of_stores_supermarket_medium,number_of_stores_supermarket_small,number_of_stores_sushi,number_of_stores_traditional_restaurant
0,2019-01,6,107,4,9,0,23,0,1,43,...,2,3,6,1,15,0,17,0,33,59
1,2019-02,5,103,1,10,3,27,0,0,80,...,2,2,15,2,17,0,14,0,33,60
2,2019-03,6,154,1,19,5,31,0,1,114,...,3,1,34,5,24,0,18,3,44,74
3,2019-04,7,190,2,17,6,45,0,1,150,...,4,1,38,7,29,0,8,7,40,84
4,2019-05,11,244,2,24,10,68,0,2,175,...,2,1,39,13,45,0,16,32,45,99
